# Analysing top N topics for top N users

In [1]:
import os
from os.path import join
import eland as ed
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

project_dir = join(os.getcwd(), os.pardir)
raw_dir = join(project_dir, 'data', 'raw')
interim_dir = join(project_dir, 'data', 'interim')

%config InlineBackend.figure_format = 'svg'

In [2]:
ed_df = ed.DataFrame('localhost', 'twitter', columns=['user_id', 'verified', 'name', 'location', 'retweet_count', 'followers_count'])

# defining the full-text query we need: Retrieving records for full_text_processed with the condition is_retweet=False and is_quote_status=False

query_unique = {
    "bool": {
        "must": {
            "term":{"is_retweet":"false"},
        },
        "filter": {
            "term":{"is_quote_status":"false"}
        },
    }
}
# using full-text search capabilities with Eland:
df_ed = ed_df.es_query(query_unique)
df_users = df_ed.to_pandas()

In [3]:
df_users = df_users.reset_index().rename(columns={'index':'tweet_id'})

In [4]:
df_users.head()

,tweet_id,user_id,verified,name,location,retweet_count,followers_count
0,1264160647002103808,1256622599364214786,False,The Meraaki,"Ahmadabad City, India",0,9
1,1264160609668599808,1251934220345208832,False,Newspapers,Dhaka,0,80
2,1264121161589415936,1251934220345208832,False,Newspapers,Dhaka,0,80
3,1264160569315209216,1113075640499036160,False,netvani,None,0,12
4,1264114187346874368,1113075640499036160,False,netvani,None,0,12


## Top N users with most original tweets

In [5]:
N = 50
top_users = df_users.groupby('name')\
    .size()\
    .to_frame()\
    .reset_index()\
    .rename(columns={0:'Count', 'name':'name'})\
    .set_index('name').nlargest(N, 'Count')

top_users

,Count
name,
The Wealth Home - Let's start building wealth,2439
MEDIAonINDIA,1566
Bharti Airtel India,713
ABP Ananda,491
Hindustan Times,340
Tata Sky,274
News18Bangla,201
Oneindia Bengali,200
S Newz,199


## Top N most retweeted users

In [6]:
N = 50
df_users.groupby('name')['retweet_count'].agg({'sum'}).nlargest(N, 'sum')

,sum
name,
Narendra Modi,41072
Shah Rukh Khan,15074
PMO India,11439
Priyanka Gandhi Vadra,6604
Amit Shah,4719
Norbert Elekes,4237
OpIndia.com,3871
BJP Bengal,3641
Honeypreet Insan,3639


## Aggregating the columns to create a popularity measure

In [7]:
df_users = df_users.groupby('user_id').agg({
            'name': lambda x: x.iloc[0],
            'followers_count': 'max',
            'retweet_count': 'sum',
            'verified': lambda x: x.iloc[0],
        })

### Normalising the columns using Z-Score

In [8]:
cols = list(df_users.columns)
cols.remove('name')

for col in cols:
    col_zscore = col + '_zscore'
    df_users[col_zscore] = (df_users[col] - df_users[col].mean())/df_users[col].std(ddof=0)

## Popularity Measure = Sum of Z-Scores (to be refined)

In [9]:
df_users['popularity'] = df_users['followers_count_zscore']+df_users['retweet_count_zscore']+df_users['verified_zscore']

In [10]:
N = 50
df_users.nlargest(N, 'popularity')

,name,followers_count,retweet_count,verified,followers_count_zscore,retweet_count_zscore,verified_zscore,popularity
user_id,,,,,,,,
18839785,Narendra Modi,60652395,41072,True,78.552852,179.297914,4.338553,262.189319
101311381,Shah Rukh Khan,40859499,15074,True,52.900567,65.782254,4.338553,123.021373
471741741,PMO India,37731486,11439,True,48.846552,49.910670,4.338553,103.095775
428333,CNN Breaking News,58390960,119,True,75.621953,0.483896,4.338553,80.444402
759251,CNN,49342437,532,True,63.894751,2.287187,4.338553,70.520491
807095,The New York Times,47070738,584,True,60.950550,2.514236,4.338553,67.803339
5402612,BBC Breaking News,44799065,521,True,58.006382,2.239158,4.338553,64.584093
1447949844,Amit Shah,21985085,4719,True,28.438666,20.568981,4.338553,53.346200
742143,BBC News (World),28654283,700,True,37.082180,3.020730,4.338553,44.441462


In [11]:
top_users = df_users.nlargest(N, 'popularity')

## Extracting Top N topics for Top N users

In [12]:
df_topics_lda = pd.read_csv(join(interim_dir, 'tweet_topics_data_lda.csv'), index_col=0) ## For LDA topics
df_topics_nmf = pd.read_csv(join(interim_dir, 'tweet_topics_data_nmf.csv'), index_col=0) ## For NMF topics

In [13]:
df_topics_lda = df_topics_lda[['user_id', 'name', 'verified', 'Topic']]
df_topics_nmf = df_topics_nmf[['user_id', 'name', 'verified', 'Topic']]

In [14]:
N = 5  ## Up to top 5 topics for every user
topics_users_lda = []
topics_users_nmf = []

for index, row in top_users.iterrows():
    topics_lda = df_topics_lda[df_topics_lda['user_id']==int(index)]
    topics_list_lda = topics_lda['Topic'].value_counts().nlargest(N).index.tolist()
    topics_users_lda.append(topics_list_lda)

    topics_nmf = df_topics_nmf[df_topics_nmf['user_id']==int(index)]
    topics_list_nmf = topics_nmf['Topic'].value_counts().nlargest(N).index.tolist()
    topics_users_nmf.append(topics_list_nmf)


In [15]:
top_users['topics_lda'] = topics_users_lda
top_users['topics_nmf'] = topics_users_nmf

In [16]:
top_users

,name,followers_count,retweet_count,verified,followers_count_zscore,retweet_count_zscore,verified_zscore,popularity,topics_lda,topics_nmf
user_id,,,,,,,,,,
18839785,Narendra Modi,60652395,41072,True,78.552852,179.297914,4.338553,262.189319,"[4, 7, 5, 1]","[7, 2]"
101311381,Shah Rukh Khan,40859499,15074,True,52.900567,65.782254,4.338553,123.021373,[2],"[7, 2]"
471741741,PMO India,37731486,11439,True,48.846552,49.910670,4.338553,103.095775,[8],[7]
428333,CNN Breaking News,58390960,119,True,75.621953,0.483896,4.338553,80.444402,[4],[10]
759251,CNN,49342437,532,True,63.894751,2.287187,4.338553,70.520491,"[4, 10, 7]","[10, 7, 2, 8]"
807095,The New York Times,47070738,584,True,60.950550,2.514236,4.338553,67.803339,"[5, 10, 7]","[2, 10]"
5402612,BBC Breaking News,44799065,521,True,58.006382,2.239158,4.338553,64.584093,[5],[9]
1447949844,Amit Shah,21985085,4719,True,28.438666,20.568981,4.338553,53.346200,"[9, 8]","[7, 5]"
742143,BBC News (World),28654283,700,True,37.082180,3.020730,4.338553,44.441462,"[5, 1, 4]","[2, 9, 8]"
